teste auto save

In [8]:
!pip install pandas


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
     ---------------------------------------- 0.0/509.2 kB ? eta -:--:--
     ------------ ------------------------- 174.1/509.2 kB 5.3 MB/s eta 0:00:01
     -------------------------------------- 509.2/509.2 kB 8.0 MB/s eta 0:00:00
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     -- ------------------------------------- 0.7/12.9 MB 21.1 MB/s eta 0:00:01
     ---- ----------------------------------- 1.5/12.9 MB 18.7 MB/s eta 0:00:01
     ------- -------------------------------- 2.5/12.9 MB 22.3 MB/s eta 0:00:01
     ---------- ----------------------------- 3.4/12.9 MB 24.2 MB/s eta 0:00:01
     -------------- ------------------------- 4.8/12.9 MB 25.7 MB/s eta 0:00:01
     -------------------- ------------------- 6.5/12.9 MB 29.6 MB/s eta 0:00:01
     ------------------------- -------------- 8.4/12.9 MB 31.4 MB/s eta 0:00:0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import requests
url = "https://dados.gov.br/"
response = requests.get(url)
print(response.status_code)

200


In [2]:
import requests
url = "https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/novo-bolsa-familia/202401_NovoBolsaFamilia.zip"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)
print(response.status_code)

200


In [6]:
import os
import requests

url = "https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/novo-bolsa-familia/202401_NovoBolsaFamilia.zip"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
caminho_base = os.path.join("G:/DEV/data_science", "dados_bolsa_familia")
os.makedirs(caminho_base, exist_ok=True)
nome_arquivo = os.path.join(caminho_base, "202401_NovoBolsaFamilia.zip")

response = requests.get(url, headers=headers)
if response.status_code == 200:
    with open(nome_arquivo, 'wb') as f:
        f.write(response.content)
    print(f"Arquivo salvo em: {nome_arquivo}")
else:
    print(f"Erro: {response.status_code}")


Arquivo salvo em: G:/DEV/data_science\dados_bolsa_familia\202401_NovoBolsaFamilia.zip


In [7]:
import zipfile


def descompactar_csv_do_zip(caminho_zip, pasta_destino):
    os.makedirs(pasta_destino, exist_ok=True)
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        arquivos_zip = zip_ref.namelist()
        arquivos_csv = [f for f in arquivos_zip if f.lower().endswith('.csv')]
        if not arquivos_csv:
            print("Nenhum arquivo CSV encontrado no ZIP.")
            return
        for arquivo_csv in arquivos_csv:
            zip_ref.extract(arquivo_csv, pasta_destino)
            print(f"{arquivo_csv} extraído para {pasta_destino}")


caminho_zip = "G:/DEV/data_science/dados_bolsa_familia/202401_NovoBolsaFamilia.zip"
pasta_destino = "G:/DEV/data_science/dados_bolsa_familia/csv_extracao"

descompactar_csv_do_zip(caminho_zip, pasta_destino)



202401_NovoBolsaFamilia.csv extraído para G:/DEV/data_science/dados_bolsa_familia/csv_extracao


In [11]:
import pandas as pd
import os

def tratar_e_salvar_parquet(caminho_csv):
    bf = pd.read_csv(caminho_csv, sep=';', encoding='latin1')
    colunas_remover = ['CPF FAVORECIDO', 'NIS FAVORECIDO', 'NOME FAVORECIDO']
    bf = bf.drop(columns=[col for col in colunas_remover if col in bf.columns])
    bf.columns = [
        'mes_competencia',
        'mes_referencia',
        'uf',
        'codigo_municipio',
        'nome_municipio',
        'valor_parcela'
    ]
    bf['mes_competencia'] = pd.to_datetime(bf['mes_competencia'].astype(str) + '01', format='%Y%m%d')
    bf['mes_referencia'] = pd.to_datetime(bf['mes_referencia'].astype(str) + '01', format='%Y%m%d')
    bf['valor_parcela'] = bf['valor_parcela'].str.replace(',', '.').astype(float)
    nome_base = os.path.splitext(os.path.basename(caminho_csv))[0]
    pasta_destino = os.path.dirname(caminho_csv)
    caminho_parquet = os.path.join(pasta_destino, f"{nome_base}.parquet")
    bf.to_parquet(caminho_parquet, index=False)

    print(f"salvo em: {caminho_parquet}")

    print(bf.dtypes)
    return caminho_parquet


caminho_csv = "G:/DEV/data_science/dados_bolsa_familia/csv_extracao/202401_NovoBolsaFamilia.csv"
tratar_e_salvar_parquet(caminho_csv)


salvo em: G:/DEV/data_science/dados_bolsa_familia/csv_extracao\202401_NovoBolsaFamilia.parquet
mes_competencia     datetime64[ns]
mes_referencia      datetime64[ns]
uf                          object
codigo_municipio             int64
nome_municipio              object
valor_parcela              float64
dtype: object


'G:/DEV/data_science/dados_bolsa_familia/csv_extracao\\202401_NovoBolsaFamilia.parquet'

In [12]:
#ler arquivo parquet
import pandas as pd
import os
caminho_parquet = "G:/DEV/data_science/dados_bolsa_familia/csv_extracao/202401_NovoBolsaFamilia.parquet"
bf_parquet = pd.read_parquet(caminho_parquet)
bf_parquet


,mes_competencia,mes_referencia,uf,codigo_municipio,nome_municipio,valor_parcela
0,2024-01-01,2023-03-01,AC,139,RIO BRANCO,800.0
1,2024-01-01,2023-03-01,BA,3329,ANGUERA,750.0
2,2024-01-01,2023-03-01,BA,3731,MONTE SANTO,650.0
3,2024-01-01,2023-03-01,BA,3761,NOVA VICOSA,650.0
4,2024-01-01,2023-03-01,BA,3849,SALVADOR,850.0
...,...,...,...,...,...,...
20833374,2024-01-01,2024-01-01,TO,9643,XAMBIOA,600.0
20833375,2024-01-01,2024-01-01,TO,9643,XAMBIOA,600.0
20833376,2024-01-01,2024-01-01,TO,9643,XAMBIOA,600.0
20833377,2024-01-01,2024-01-01,TO,9643,XAMBIOA,440.0


In [10]:
caminho_csv = "G:/DEV/data_science/dados_bolsa_familia/csv_extracao/202401_NovoBolsaFamilia.csv"
bf_csv = pd.read_csv(caminho_csv, sep=';', encoding='latin1')
bf_csv

,MÊS COMPETÊNCIA,MÊS REFERÊNCIA,UF,CÓDIGO MUNICÍPIO SIAFI,NOME MUNICÍPIO,CPF FAVORECIDO,NIS FAVORECIDO,NOME FAVORECIDO,VALOR PARCELA
0,202401,202303,AC,139,RIO BRANCO,***.511.972-**,1.629112e+10,JANAINA FERREIRA DA SILVA,"800,00"
1,202401,202303,BA,3329,ANGUERA,***.909.745-**,2.047443e+10,TACIANE MARQUES DA SILVA,"750,00"
2,202401,202303,BA,3731,MONTE SANTO,***.872.338-**,2.019538e+10,MAILSON DOS SANTOS SILVA,"650,00"
3,202401,202303,BA,3761,NOVA VICOSA,***.665.905-**,1.272359e+10,IRENI RICARDO GERALDO,"650,00"
4,202401,202303,BA,3849,SALVADOR,***.932.435-**,2.002458e+10,ELIENE GOMES DOS SANTOS,"850,00"
...,...,...,...,...,...,...,...,...,...
20833374,202401,202401,TO,9643,XAMBIOA,NaN,2.032433e+10,ZEILA MARIA BATISTA,"600,00"
20833375,202401,202401,TO,9643,XAMBIOA,***.273.191-**,2.064488e+10,ZENILDE ALVES DOS SANTOS,"600,00"
20833376,202401,202401,TO,9643,XAMBIOA,NaN,1.905866e+10,ZENOLIA RAMOS DA SILVA CARVALHO,"600,00"
20833377,202401,202401,TO,9643,XAMBIOA,***.822.651-**,2.091926e+10,ZILMA ALVES DE BARROS,"440,00"
